# Diffusion - Image generation with diffusion models

In [ ]:
# Install dependencies
! pip install numpy
! pip install matplotlib
! pip install seaborn
! pip install scikit-learn
! pip install scikit-image
! pip install scipy

! pip install tensorflow --upgrade
! pip install tensorflow-io
! pip install keras --upgrade

In [ ]:
# Please read the guidelines on https://www.kaggle.com/docs/api#authentication
# to get an idea where to put your kaggle API token to be able to download the dataset

! pip install kaggle
! kaggle datasets download -d jhoward/lsun_bedroom

! unzip lsun_bedroom.zip -d data/
! rm lsun_bedroom.zip

In [ ]:
# Imports
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import warnings

from pathlib import Path

import numpy as np
import shutil

import tensorflow as tf
import keras
from keras import layers, models
from keras.applications import InceptionV3
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from scipy import linalg
from skimage.transform import resize

import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Data Exploration

In [ ]:
data_path = Path("data/sample/data0/lsun/bedroom")
keras.utils.set_random_seed(42)

In [ ]:
dataset_variables = {  # Interesting parameters I might want to change later
    # One of "grayscale", "rgb", "rgba". Defaults to "rgb". Whether the images will be converted to have 1, 3, or 4 channels.
    "color_mode": "rgb",
    # Size of the batches of data. Defaults to 32. If None, the data will not be batched (the dataset will yield individual samples).
    "batch_size": 32,
    # Size to resize images to after they are read from disk, specified as (height, width). Defaults to (256, 256).
    "image_size": (256, 256),
    "labels": None,  # no labels
    # Whether to shuffle the data. Defaults to True. If set to False, sorts the data in alphanumeric order.
    "shuffle": True
}
dataset = keras.utils.image_dataset_from_directory(
    data_path,
    **dataset_variables,
)

## Helper functions

## Fréchet Inception Distance (FID)
Introduced by Heusel et al. (https://arxiv.org/abs/1706.08500). We adapt their reference implementation which is available at https://github.com/bioinf-jku/TTUR/blob/master/fid.py. We also use code from https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/.

In [ ]:
# From Heusel et al.
# For computing the FID, we propagated all images from the training dataset through the pretrained
# Inception-v3 model following the computation of the Inception Score [53], however, we use the last
# pooling layer as coding layer. For this coding layer, we calculated the mean mw and the covariance matrix Cw.
inception_input_shape = (75, 75, 3)
inception_model = InceptionV3(
    include_top=False, pooling='avg', input_shape=inception_input_shape)

In [ ]:
# input preprocessing
def frechet_image_preprocessing(images):
    prepressed = keras.applications.inception_v3.preprocess_input(
        images.astype("float32"))
    resized = np.asarray(
        [resize(image, inception_input_shape, 0) for image in prepressed])
    return resized

def get_mu_sigma(prediction):
    mu =  prediction.mean(axis=0)
    sigma = np.cov(prediction, rowvar=False)
    return mu, sigma
    

In [ ]:
# Function inspired from https://machinelearningmastery.com/how-to-implement-the-frechet-inception-distance-fid-from-scratch/
def frechet_distance(images1, images2, model):
    # calculate activations
    pred1 = model.predict(frechet_image_preprocessing(images1))
    mu1, sigma1 = get_mu_sigma(pred1)
    pred2 = model.predict(frechet_image_preprocessing(images2))
    mu2, sigma2 = get_mu_sigma(pred2)
    
    return calculate_frechet_distance(mu1, sigma1, mu2, sigma2)

# From https://github.com/bioinf-jku/TTUR/blob/master/fid.py
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).

    Stable version by Dougal J. Sutherland.

    Params:
    -- mu1 : Numpy array containing the activations of the pool_3 layer of the
             inception net ( like returned by the function 'get_predictions')
             for generated samples.
    -- mu2   : The sample mean over activations of the pool_3 layer, precalcualted
               on an representive data set.
    -- sigma1: The covariance matrix over activations of the pool_3 layer for
               generated samples.
    -- sigma2: The covariance matrix over activations of the pool_3 layer,
               precalcualted on an representive data set.

    Returns:
    --   : The Frechet Distance.
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, "Training and test mean vectors have different lengths"
    assert sigma1.shape == sigma2.shape, "Training and test covariances have different dimensions"

    diff = mu1 - mu2

    # product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = "fid calculation produces singular product; adding %s to diagonal of cov estimates" % eps
        warnings.warn(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError("Imaginary component {}".format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean

Note: From the properties of the dot product, we know that the dot product of a vector with itself if the squared euclidian norm (https://math.libretexts.org/Bookshelves/Calculus/Calculus_(OpenStax)/12%3A_Vectors_in_Space/12.03%3A_The_Dot_Product#Properties_of_the_Dot_Product).